In [1]:
%pylab inline
import pandas as pd
import scanpy as sc

In [2]:
def postprocess(adata,view=True):
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata)
    adata.raw = adata
    adata = adata[:, adata.var.highly_variable]
    print('after select HVG',adata.shape)
    sc.pp.scale(adata)
    sc.tl.pca(adata)
    sc.pp.neighbors(adata)
    if view:
        sc.tl.tsne(adata)
        sc.tl.umap(adata)
    return adata

In [3]:
def cluster_k_leiden(embadata,n_cluster):
    max_steps=120
    this_step = 0
    this_min = 0
    this_max = 2
    print('reference cluster number',n_cluster)
    while this_step < max_steps:
        this_resolution = this_min + ((this_max-this_min)/2)
        sc.tl.leiden(embadata,resolution=this_resolution,random_state=42)
        this_clusters = embadata.obs['leiden'].nunique()
        if this_clusters > n_cluster:
            this_max = this_resolution
        elif this_clusters < n_cluster:
            this_min = this_resolution
        else:break
        this_step+=1
    if this_step==max_steps:
        print('Cannot find the number of clusters')
        print('Use resolution',this_resolution)
    else:
        print('use resolution',this_resolution)
     # leiden
    sc.tl.leiden(embadata,resolution=this_resolution,random_state=42,key_added=f'cluster_{n_cluster}')

In [4]:
refdf = pd.read_csv('./baron/baron_human_ref_19264_fromsaver.csv',index_col=0)
sampledf = pd.read_csv('./baron/baron_human_samp_19264_fromsaver.csv',index_col=0)
magicdf = pd.read_csv('./baron/baron_human_magic.csv',index_col=0,sep='\t').T
saverdf = pd.read_csv('./baron/baron_human_saver.csv',index_col=0,sep='\t').T

In [5]:
scimputedf = pd.read_csv('./SAVER-data/baron_human_samp_scimpute.csv',index_col=0,sep='\t').T

In [6]:
# ref
import scanpy as sc
tmp2adata = sc.AnnData(refdf)
sc.pp.calculate_qc_metrics(tmp2adata,percent_top=None, log1p=False, inplace=True)
figsize(4,3)
sc.pl.violin(tmp2adata, ['n_genes_by_counts', 'total_counts'],
             jitter=0.4, multi_panel=True)

In [8]:
# ref
import scanpy as sc
tmp2adata = sc.AnnData(sampledf)
sc.pp.calculate_qc_metrics(tmp2adata,percent_top=None, log1p=False, inplace=True)
figsize(4,3)
sc.pl.violin(tmp2adata, ['n_genes_by_counts', 'total_counts'],
             jitter=0.4, multi_panel=True)

In [9]:
refAdata = sc.AnnData(refdf)
refAdata = postprocess(refAdata,True)

sampleAdata = sc.AnnData(sampledf)
sampleAdata = postprocess(sampleAdata,True)

magicAdata = sc.AnnData(magicdf)
magicAdata = postprocess(magicAdata,True)

saverAdata = sc.AnnData(saverdf)
saverAdata = postprocess(saverAdata,True)

scimputeAdata = sc.AnnData(scimputedf)
scimputeAdata = postprocess(scimputeAdata,True)

In [10]:
sc.tl.leiden(refAdata,resolution=0.6)

In [11]:
numcls = refAdata.obs.leiden.unique().shape[0]
numcls

In [12]:
sc.pl.umap(refAdata,color=[f'leiden'],size=25,title='Reference',legend_loc='on data')

In [13]:
sampledf = pd.read_csv('./baron/baron_human_samp_19264_fromsaver.csv',index_col=0)
scviAdata = sc.AnnData(sampledf)

sc.pp.highly_variable_genes(
    scviAdata,
    flavor="seurat_v3",
    n_top_genes=2000,
    subset=True,
)

import scvi
scvi.model.SCVI.setup_anndata(scviAdata)

vae = scvi.model.SCVI(scviAdata)

vae.train()

scviemb = vae.get_latent_representation()

scviAdata = sc.AnnData(pd.DataFrame(scviemb,index=refdf.index))
sc.pp.scale(scviAdata)
sc.tl.pca(scviAdata)
sc.pp.neighbors(scviAdata)

sc.tl.umap(scviAdata)
sc.tl.tsne(scviAdata)

In [14]:
cluster_k_leiden(sampleAdata,numcls)
cluster_k_leiden(magicAdata,numcls)
cluster_k_leiden(saverAdata,numcls)
cluster_k_leiden(scimputeAdata,numcls)
cluster_k_leiden(scviAdata,numcls)

In [15]:
adatalist = [sampleAdata,magicAdata,saverAdata,scimputeAdata,scviAdata]
for tmpadata in adatalist:
    tmpadata.obs['refleiden']=refAdata.obs['leiden']

## continous

In [16]:
from sklearn.metrics.cluster import adjusted_rand_score,normalized_mutual_info_score,davies_bouldin_score,calinski_harabasz_score,silhouette_score

scnmi=[]
scari=[]
scch = []
scdb=[]
scsil=[]
for i in np.arange(1,5.5,0.5):
    if i==0:
        # imputeemb = np.load('baron/baron_human_samp_19264_fromsaver_50M-0.1B-res_embedding_768.npy')
        imputeemb = np.load('baron/baron_human_samp_19264_fromsaver_50M-0.1B-res_embedding.npy')
        
    else:
        # imputeemb = np.load(f'baron/baron_human_samp_19264_fromsaver_50M-0.1B-res_tgthighres{i:.1f}_embedding_768.npy')
        imputeemb = np.load(f'baron/baron_human_samp_19264_fromsaver_50M-0.1B-res_fold{i:.1f}_embedding.npy')
        
    imputeAdata = sc.AnnData(pd.DataFrame(imputeemb,index=refdf.index))
    sc.pp.scale(imputeAdata)
    sc.tl.pca(imputeAdata)
    sc.pp.neighbors(imputeAdata)
    # sc.tl.tsne(imputeAdata)
    # sc.tl.umap(imputeAdata)
    cluster_k_leiden(imputeAdata,numcls)    
    imputenmi = normalized_mutual_info_score(refAdata.obs[f'leiden'], imputeAdata.obs[f'cluster_{numcls}']) 
    imputeari = adjusted_rand_score(refAdata.obs[f'leiden'], imputeAdata.obs[f'cluster_{numcls}']) 
    scnmi.append(imputenmi)
    scari.append(imputeari)
    scch.append(calinski_harabasz_score(refAdata.obsm['X_pca'],imputeAdata.obs[f'cluster_{numcls}']))
    scdb.append(davies_bouldin_score(refAdata.obsm['X_pca'],imputeAdata.obs[f'cluster_{numcls}']))
    scsil.append(silhouette_score(refAdata.obsm['X_pca'],imputeAdata.obs[f'cluster_{numcls}']))

### NMI

In [17]:
import seaborn as sns
import colorbm as cbm
sns.set_palette(sns.color_palette(cbm.pal('npg').as_hex))
rcParams['axes.spines.right'] = False
rcParams['axes.spines.top'] = False
rcParams['axes.spines.left'] = True
rcParams['axes.spines.bottom'] = True
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42

In [18]:
figsize(3,2)
import seaborn as sns
scorelist=[]
for tmpadata in adatalist:
    scorelist.append(normalized_mutual_info_score(tmpadata.obs[f'refleiden'], tmpadata.obs[f'cluster_{numcls}']))

axisx = np.arange(1,5.5,0.5)
    
sns.scatterplot(x=axisx,y=scnmi)
sns.lineplot(x=axisx,y=scnmi,label=f'scEPT')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[0]]*2,label=f'Sample',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[1]]*2,label=f'MAGIC',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[2]]*2,label=f'SAVER',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[3]]*2,label=f'scImpute',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[4]]*2,label=f'scVI',linestyle='dashed')
plt.legend(loc='center right',bbox_to_anchor=(1.4, 0.5))
plt.legend([],[], frameon=False)
plt.title(f"NMI")
plt.xlabel(f"Fold");
# plt.savefig('Baron_NMI_fold.pdf',bbox_inches='tight')

### ARI

In [19]:
figsize(3,2)
scorelist=[]
for tmpadata in adatalist:
    scorelist.append(adjusted_rand_score(tmpadata.obs[f'refleiden'], tmpadata.obs[f'cluster_{numcls}']))

sns.scatterplot(x=axisx,y=scari)
sns.lineplot(x=axisx,y=scari,label=f'scEPT Highres')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[0]]*2,label=f'Sample',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[1]]*2,label=f'MAGIC',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[2]]*2,label=f'SAVER',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[3]]*2,label=f'scImpute',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[4]]*2,label=f'scVI',linestyle='dashed')

plt.legend(loc='center right',bbox_to_anchor=(1.4, 0.5))
plt.legend([],[], frameon=False)
plt.title(f"ARI")
# plt.xlabel(f"Added Resolution (Log)");
plt.xlabel(f"Fold");
# plt.savefig('Baron_ARI_fold.pdf',bbox_inches='tight')

### silhouette_score

In [20]:
figsize(3,2)
scorelist=[]
for tmpadata in adatalist:
    scorelist.append(silhouette_score(refAdata.obsm[f'X_pca'], tmpadata.obs[f'cluster_{numcls}']))
# scorelist.append(silhouette_score(adatalist[-1].X, adatalist[-1].obs[f'cluster_{numcls}']))

sns.scatterplot(x=axisx,y=scsil)
sns.lineplot(x=axisx,y=scsil,label=f'scEPT Highres')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[0]]*2,label=f'Sample',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[1]]*2,label=f'MAGIC',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[2]]*2,label=f'SAVER',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[3]]*2,label=f'scImpute',linestyle='dashed')
sns.lineplot(x=[axisx[0],axisx[-1]],y=[scorelist[4]]*2,label=f'scVI',linestyle='dashed')

plt.legend(loc='center right',bbox_to_anchor=(1.4, 0.5))
plt.legend([],[], frameon=False)
plt.title(f"Silhouette Score")
# plt.xlabel(f"Added Resolution (Log)");
plt.xlabel(f"Fold");
# plt.savefig('Baron_sil_fold.pdf',bbox_inches='tight')

In [21]:
imputeemb = np.load(f'baron/baron_human_samp_19264_fromsaver_50M-0.1B-res_tgthighres5_embedding.npy')
imputeAdata = sc.AnnData(pd.DataFrame(imputeemb,index=refdf.index))
sc.pp.scale(imputeAdata)
sc.tl.pca(imputeAdata)
sc.pp.neighbors(imputeAdata)

sc.tl.tsne(imputeAdata)
sc.tl.umap(imputeAdata)

cluster_k_leiden(imputeAdata,numcls)
imputeAdata.obs['refleiden']=refAdata.obs['leiden']

In [23]:
namelist = ['Sample','MAGIC','SAVER','scImpute','scVI','scFoundation']

In [28]:
sns.set_palette(sns.color_palette(cbm.pal('npg').as_hex))
fig, axes = plt.subplots(2,7,figsize=(25,6))
fig.suptitle(f'Baron human dataset from SAVER',fontsize=20)

sc.pl.umap(refAdata,color=[f'leiden'],size=25,ax=axes[0][0],show=False,title='Ref cluster',legend_loc='on data',legend_fontweight='black',legend_fontsize='large',frameon=False)
sc.pl.umap(sampleAdata,color=[f'leiden'],size=25,ax=axes[0][1],show=False,title='Sample',legend_loc='on data',palette=refAdata.uns['leiden_colors'],legend_fontweight='black',legend_fontsize='x-large',frameon=True)
sc.pl.umap(magicAdata,color=[f'leiden'],size=25,ax=axes[0][2],show=False,title='Magic',legend_loc='on data',palette=refAdata.uns['leiden_colors'],legend_fontweight='black',legend_fontsize='x-large',frameon=True)
sc.pl.umap(saverAdata,color=[f'leiden'],size=25,ax=axes[0][3],show=False,title='Saver',legend_loc='on data',palette=refAdata.uns['leiden_colors'],legend_fontweight='black',legend_fontsize='x-large',frameon=True)
sc.pl.umap(scimputeAdata,color=[f'leiden'],size=25,ax=axes[0][4],show=False,title='scImpute',legend_loc='on data',palette=refAdata.uns['leiden_colors'],legend_fontweight='black',legend_fontsize='x-large',frameon=True)
sc.pl.umap(scviAdata,color=[f'leiden'],size=25,ax=axes[0][5],show=False,title='scVI',legend_loc='on data',palette=refAdata.uns['leiden_colors'],legend_fontweight='black',legend_fontsize='x-large',frameon=True)
sc.pl.umap(imputeAdata,color=[f'leiden'],size=25,ax=axes[0][6],show=False,title='scEPT',legend_loc='on data',palette=refAdata.uns['leiden_colors'],legend_fontweight='black',legend_fontsize='x-large',frameon=True)


sc.pl.umap(refAdata,color=[f'leiden'],size=25,ax=axes[1][0],show=False,title='Reference',legend_loc='on data')
for name,tmpadata in zip(namelist,adatalist):
    refAdata.obs['tmpcls']=tmpadata.obs[f'cluster_{numcls}']
    sc.pl.umap(refAdata,color=[f'tmpcls'],size=25,ax=axes[1][namelist.index(name)+1],show=False,title=name,legend_loc='on data',legend_fontweight='black',legend_fontsize='x-large',frameon=True,palette=refAdata.uns['leiden_colors'])
sc.pl.umap(refAdata,color=[f'leiden'],size=25,ax=axes[1][6],show=False,title='Reference',legend_loc='on data',frameon=True,palette=refAdata.uns['leiden_colors'])